In [225]:
import pandas as pd
import constants as const
import numpy as np

In [255]:
# df1 = pd.read_csv(f'./data/extracted/thoi_trang_raw.csv', index_col=0)
df1 = pd.read_csv(f'./data/extracted/do_dien_tu_raw.csv', index_col=0)

#### replace none, empty data by nan value


In [256]:
df1.replace({'None': np.nan, '': np.nan}, inplace=True)

In [257]:
df1.isnull().sum()

Product Name          0
Price                10
Brand              3624
Categories            0
Product Sold          0
Stock               308
Rating                0
5 Start               0
4 Start               0
3 Start               0
2 Start               0
1 Start               0
Total Rating          0
No of Comments        0
Image and Video       0
dtype: int64

# Data Cleaning

#### Clean rating

In [258]:
df1["5 Start"] = df1['5 Start'].str.extract(r'(\([,.\d]*k*\))')
df1["5 Start"] = df1['5 Start'].replace({',':'.'}, regex=True).replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)

df1["4 Start"] = df1['4 Start'].str.extract(r'(\([,.\d]*k*\))')
df1["4 Start"] = df1['4 Start'].replace({',':'.'}, regex=True).replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)

df1["3 Start"] = df1['3 Start'].str.extract(r'(\([,.\d]*k*\))')
df1["3 Start"] = df1['3 Start'].replace({',':'.'}, regex=True).replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)

df1["2 Start"] = df1['2 Start'].str.extract(r'(\([,.\d]*k*\))')
df1["2 Start"] = df1['2 Start'].replace({',':'.'}, regex=True).replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)

df1["1 Start"] = df1['1 Start'].str.extract(r'(\([,.\d]*k*\))')
df1["1 Start"] = df1['1 Start'].replace({',':'.'}, regex=True).replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)

df1['Rating'] = df1['Rating'].astype(float)

In [259]:
def calc_rating(row):
    total = row['5 Start'] + row['4 Start'] + row['3 Start'] + row['2 Start'] + row['1 Start']
    rate = 0
    if total > 0:
        rate = round((5*row['5 Start'] + 4*row['4 Start'] + 3*row['3 Start'] + 2*row['2 Start'] + 1*row['1 Start'])/total,1)
    return rate

In [260]:
# df1['Rating'] = df1['Rating'].apply(lambda row:  if type(row) is not float else np.nan)
df1['Rating'] = df1.apply(calc_rating, axis=1)

In [261]:
df1["Total Rating"] = df1['Total Rating'].replace({',':'.'}, regex=True).replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)

In [262]:
df1["No of Comments"] = df1['No of Comments'].str.extract(r'(\([,.\d]*k*\))')
df1["Image and Video"] = df1['Image and Video'].str.extract(r'(\([,.\d]*k*\))')

In [263]:
df1["No of Comments"] = df1['No of Comments'].replace({',':'.'}, regex=True).replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)
df1["Image and Video"] = df1['Image and Video'].replace({',':'.'}, regex=True).replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)

### Clean Brand

In [264]:
brands = df1['Brand'].unique()

In [265]:
def lookupBrand(name):
  return next((x for x in brands if str(x).lower().strip() in name.lower()), np.nan)

In [266]:
df1['Brand'] = df1.apply(lambda x: x['Brand'] if not pd.isnull(x['Brand']) else lookupBrand(x['Product Name']), axis=1)
df1['Brand'] = df1.apply(lambda x: x['Brand'] if not pd.isnull(x['Brand']) else lookupBrand(x['Categories']), axis=1)
df1['Brand'].fillna(method='ffill')

0          NEC
1           HP
2         Asus
3         Asus
4           HP
         ...  
6784    Xiaomi
6785    ZOLELE
6786      YILI
6787     Midea
6788    deerma
Name: Brand, Length: 6789, dtype: object

#### Product Name - Remove Unknown Character 


In [267]:
import re
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [268]:
df1['Product Name'] = df1['Product Name'].apply(lambda x: deEmojify(x))
# df1['Product Name'] = df1['Product Name'].str.replace(r'[^\x00-\x7F]+', '')

#### Price

In [269]:
def average(lst):
  return round(sum(lst) / len(lst), 2)

In [270]:
df1['Price'] = df1['Price'].str.replace('₫', '').str.replace('.', '').str.split('-').apply(lambda row: average([float(x.strip(' ')) for x in row]) if type(row) is not float else np.nan)


/var/folders/y8/snwjrj_d75v27837y00p3vk80000gn/T/ipykernel_23351/2955519610.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df1['Price'] = df1['Price'].str.replace('₫', '').str.replace('.', '').str.split('-').apply(lambda row: average([float(x.strip(' ')) for x in row]) if type(row) is not float else np.nan)


In [271]:
df1["Price"].fillna((df1["Price"].mean()),inplace = True)
df1["Price"] = df1["Price"].astype("float").round(2)

### Product Sold and stock

In [272]:
df1["Product Sold"] = df1['Product Sold'].str.extract(r'(\d+[.,\d]*k*)')
# df1['Product Sold'].fillna((df1['Product Sold'].mean()), inplace=True)

In [273]:
df1["Product Sold"] = df1['Product Sold'].replace({',':'.'}, regex=True).replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)


In [274]:
df1['Product Sold'].fillna((df1['Product Sold'].mean()), inplace=True)

In [275]:
df1['Stock'] = df1['Stock'].astype(float)
df1['Stock'].interpolate(method='linear')
df1['Stock'].fillna((round(df1['Stock'].mean(),0)), inplace=True)


#### Get parent category

In [276]:
df1['Parent Category'] = df1['Categories'].str.split('/').str[0]

In [277]:
df1.fillna(method='ffill', inplace=True)

In [278]:
df1.dtypes

Product Name        object
Price              float64
Brand               object
Categories          object
Product Sold         int64
Stock              float64
Rating             float64
5 Start              int64
4 Start              int64
3 Start              int64
2 Start              int64
1 Start              int64
Total Rating         int64
No of Comments       int64
Image and Video      int64
Parent Category     object
dtype: object

In [279]:
df1.isnull().sum()

Product Name       0
Price              0
Brand              0
Categories         0
Product Sold       0
Stock              0
Rating             0
5 Start            0
4 Start            0
3 Start            0
2 Start            0
1 Start            0
Total Rating       0
No of Comments     0
Image and Video    0
Parent Category    0
dtype: int64

In [280]:
df1["Product Name"] = df1["Product Name"].astype('string')
df1["Brand"] = df1["Brand"].astype('string')
df1["Categories"] = df1["Categories"].astype('string')
df1["Parent Category"] = df1["Parent Category"].astype('string')
df1["Stock"] = df1["Stock"].astype(int)


In [281]:
df1.dtypes

Product Name        string
Price              float64
Brand               string
Categories          string
Product Sold         int64
Stock                int64
Rating             float64
5 Start              int64
4 Start              int64
3 Start              int64
2 Start              int64
1 Start              int64
Total Rating         int64
No of Comments       int64
Image and Video      int64
Parent Category     string
dtype: object

## Export to file

In [282]:
df1.to_csv('./data/extracted/do_dien_tu.csv')